Preprocessing

In [3]:
# Load dataset
def load_data():
    data = pd.read_csv('jokowi.csv')#ubah nama file sesai dengan nama file kalian
    return data

tweet_df = load_data()
tweet_df.head()

,datetime,username,content
0,2022-09-07 23:59:58+00:00,young_ladylike,@erikamelfa @jokowi Parah emang...\nMalah bila...
1,2022-09-07 23:59:54+00:00,SastroJoyo1,@Rizmaya__ @jokowi @PDemokrat @SBYudhoyono O.....
2,2022-09-07 23:59:47+00:00,oketa214,@jokowi Bangsa Indonesia dengan bapak Jokowi t...
3,2022-09-07 23:59:42+00:00,pahlevi0884,Berkolaborasi dengan tiga kementrian tingkatka...
4,2022-09-07 23:59:32+00:00,_ikhwanyan,@tvOneNews Pemerintah @jokowi tidak serius lak...


In [4]:
# Load dataset
def load_data():
    data = pd.read_csv('jokowi.csv')#ubah nama file sesai dengan nama file kalian
    return data

tweet_df = load_data()
tweet_df.head()


,datetime,username,content
0,2022-09-07 23:59:58+00:00,young_ladylike,@erikamelfa @jokowi Parah emang...\nMalah bila...
1,2022-09-07 23:59:54+00:00,SastroJoyo1,@Rizmaya__ @jokowi @PDemokrat @SBYudhoyono O.....
2,2022-09-07 23:59:47+00:00,oketa214,@jokowi Bangsa Indonesia dengan bapak Jokowi t...
3,2022-09-07 23:59:42+00:00,pahlevi0884,Berkolaborasi dengan tiga kementrian tingkatka...
4,2022-09-07 23:59:32+00:00,_ikhwanyan,@tvOneNews Pemerintah @jokowi tidak serius lak...


In [5]:
#definisi dataframe
df  = pd.DataFrame(tweet_df[['datetime','username', 'content']])

In [6]:
#remove user 
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt    
df['remove_user'] = np.vectorize(remove_pattern)(df['content'], "@[\w]*")

In [7]:
#remove user
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt    
df['remove_user'] = np.vectorize(remove_pattern)(df['content'], "@[\w]*")

def remove(tweet):
    #remove angka
    tweet = re.sub('[0-9]+', '', tweet)
    
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
 
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    return tweet
df['remove_http'] = df['remove_user'].apply(lambda x: remove(x))
df.sort_values("remove_http", inplace = True)
df.drop_duplicates(subset ="remove_http", keep = 'first', inplace = True)


In [8]:
#import stopword
from nltk.corpus import stopwords 
stopwords_indonesia = stopwords.words('indonesian')
 
#import sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#tokenize
from nltk.tokenize import TweetTokenizer

In [9]:
# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
 
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)

In [10]:
def clean_tweets(tweet):
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
 
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
 
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
    #remove coma
    tweet = re.sub(r',','',tweet)
    
    #remove angka
    tweet = re.sub('[0-9]+', '', tweet)
     # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
 
    tweets_clean = []    
    for word in tweet_tokens:
        if (word not in stopwords_indonesia and # remove stopwords
              word not in emoticons and # remove emoticons
                word not in string.punctuation): # remove punctuation
            #tweets_clean.append(word)
            stem_word = stemmer.stem(word) # stemming word
            tweets_clean.append(stem_word)
 
    return tweets_clean
df['tweet_clean'] = df['remove_http'].apply(lambda x: clean_tweets(x))

In [11]:
#remove punct
def remove_punct(text):
    text  = " ".join([char for char in text if char not in string.punctuation])
    return text
df['Tweet'] = df['tweet_clean'].apply(lambda x: remove_punct(x))
 #=========================================================================#
df.sort_values("Tweet", inplace = True)
df.drop(df.columns[[0,1,2,3,4]], axis = 1, inplace = True)
df.drop_duplicates(subset ="Tweet", keep = 'first', inplace = True)
df.to_csv('output.csv',encoding='utf8', index=False)
df

,tweet_clean,Tweet
274,[],
55,"[abang, nggak, kenal, bimantara, humpuss, citr...",abang nggak kenal bimantara humpuss citra lamt...
649,"[abunawas, laen, jokowi boong, g, merugika, ra...",abunawas laen jokowi boong g merugika rakyat j...
578,"[abunawas, pas]",abunawas pas
626,"[adil, pertanggungjawaban, buka]",adil pertanggungjawaban buka
...,...,...
727,"[yth, presiden, kurang, inflasi, qbat, bbm, bb...",yth presiden kurang inflasi qbat bbm bbm tdk t...
348,"[yth, presiden, mohon, maaf, sih, urgensi, ban...",yth presiden mohon maaf sih urgensi bangun ikn...
751,"[yth, selamat, pilih, bpk, abdu, selamat, tuga...",yth selamat pilih bpk abdu selamat tugas
875,"[yuk, dkasih, ruang, utk, yg]",yuk dkasih ruang utk yg
